In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import logging
import os

# Configuração inicial do logging
LOG_FILE = "geocoding.log"
CACHE_FILE = "geocoding_cache.json"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

def setup_selenium_edge():
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")  # Executa o navegador em modo headless
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    logging.info("WebDriver session started.")
    return driver

def user_input_file():
    input_file = input("Por favor, insira o caminho do arquivo de entrada (ex: input.xlsx): ")
    return input_file

def user_select_columns(df):
    print("Colunas disponíveis no arquivo:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i}. {col}")
    selected_columns_indexes = input("Selecione as colunas de endereço pelo número, separadas por vírgula (ex: 1,2,3,4): ")
    selected_columns = [df.columns[int(index)-1] for index in selected_columns_indexes.split(',')]
    return selected_columns

def prepare_output_file(input_file):
    output_file = os.path.splitext(input_file)[0] + "_output.xlsx"
    if not os.path.exists(output_file):
        df_output = pd.read_excel(input_file)
        df_output = df_output.assign(Latitude=None, Longitude=None, Processed=False)
        df_output.to_excel(output_file, index=False)
    else:
        df_output = pd.read_excel(output_file)
    return df_output, output_file

def load_or_create_cache():
    if not os.path.isfile(CACHE_FILE):
        with open(CACHE_FILE, 'w') as file:
            json.dump({}, file)
        logging.info("New cache file created.")
    try:
        with open(CACHE_FILE, 'r') as file:
            cache = json.load(file)
    except json.JSONDecodeError:
        logging.error("Cache file is corrupted. Creating a new one.")
        cache = {}
    return cache

def update_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4)

def get_lat_long(driver, address, cache, max_retries=3):
    if address in cache:
        logging.info(f"Address '{address}' found in cache.")
        return cache[address]['latitude'], cache[address]['longitude'], True

    retries = 0
    while retries < max_retries:
        try:
            driver.get(f"https://www.google.com/maps/search/{'+'.join(address.split())}")
            wait = WebDriverWait(driver, 10)
            wait.until(EC.url_contains("@"))
            url = driver.current_url
            coords = url.split('@')[1].split(',')[0:2]
            latitude, longitude = coords[0], coords[1].split('!')[0]
            cache[address] = {'latitude': latitude, 'longitude': longitude}
            update_cache(cache)  # Atualiza o cache imediatamente após cada geocodificação bem-sucedida
            return latitude, longitude, True
        except Exception as e:
            logging.error(f"Retry {retries + 1} for address '{address}': {e}")
            retries += 1
    return None, None, False

def process_addresses(driver, df_output, cache, address_column_names, output_file):
    for index, row in df_output[df_output['Processed'] == False].iterrows():
        address_parts = [str(row[col]).strip() for col in address_column_names]
        address = ' '.join(address_parts)
        if address and not row['Processed']:
            latitude, longitude, success = get_lat_long(driver, address, cache)
            if success:
                df_output.at[index, 'Latitude'] = latitude
                df_output.at[index, 'Longitude'] = longitude
                df_output.at[index, 'Processed'] = True
                df_output.to_excel(output_file, index=False)  # Salva após cada geocodificação bem-sucedida

    return df_output

def main():
    cache = load_or_create_cache()  # Garante a criação ou carga do arquivo de cache

    input_file = user_input_file()
    df_input = pd.read_excel(input_file)
    address_column_names = user_select_columns(df_input)

    driver = setup_selenium_edge()
    df_output, output_file = prepare_output_file(input_file)

    df_output = process_addresses(driver, df_output, cache, address_column_names, output_file)
    update_cache(cache)
    logging.info("Geocoding process completed.")
    print(f"The spreadsheet has been updated. Output file: {output_file}")

if __name__ == "__main__":
    main()


Por favor, insira o caminho do arquivo de entrada (ex: input.xlsx):  Planilha com informações de Sondagens.xlsx


Colunas disponíveis no arquivo:
1. Sondagem 
2. Rua / Avenida 
3. Número 
4. Trecho 
5. Bairro 
6. CEP
7. Índice de Absorção do Solo


Selecione as colunas de endereço pelo número, separadas por vírgula (ex: 1,2,3,4):  2,3,5,6


C:\Users\jairo\AppData\Local\Temp\ipykernel_31784\1839482510.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-3.8044711' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_output.at[index, 'Latitude'] = latitude
C:\Users\jairo\AppData\Local\Temp\ipykernel_31784\1839482510.py:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-38.572465' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_output.at[index, 'Longitude'] = longitude


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import logging
import os

# Configuração inicial do logging
LOG_FILE = "geocoding.log"
CACHE_FILE = "geocoding_cache.json"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

def setup_selenium_edge():
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")  # Executa o navegador em modo headless
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    logging.info("WebDriver session started.")
    return driver

def user_input_file():
    input_file = input("Por favor, insira o caminho do arquivo de entrada (ex: input.xlsx): ")
    return input_file

def user_select_columns(df):
    print("Colunas disponíveis no arquivo:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i}. {col}")
    selected_columns_indexes = input("Selecione as colunas de endereço pelo número, separadas por vírgula (ex: 1,2,3,4): ")
    selected_columns = [df.columns[int(index)-1] for index in selected_columns_indexes.split(',')]
    return selected_columns

def prepare_output_file(input_file):
    output_file = os.path.splitext(input_file)[0] + "_output.xlsx"
    if not os.path.exists(output_file):
        df_output = pd.read_excel(input_file)
        df_output = df_output.assign(Latitude=None, Longitude=None, Processed=False)
        df_output.to_excel(output_file, index=False)
    else:
        df_output = pd.read_excel(output_file)
    return df_output, output_file

def load_or_create_cache():
    if not os.path.isfile(CACHE_FILE):
        with open(CACHE_FILE, 'w') as file:
            json.dump({}, file)
        logging.info("New cache file created.")
    try:
        with open(CACHE_FILE, 'r') as file:
            cache = json.load(file)
    except json.JSONDecodeError:
        logging.error("Cache file is corrupted. Creating a new one.")
        cache = {}
    return cache

def update_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4)

def get_lat_long(driver, address, cache, max_retries=3):
    if address in cache:
        logging.info(f"Address '{address}' found in cache.")
        return cache[address]['latitude'], cache[address]['longitude'], True

    retries = 0
    while retries < max_retries:
        try:
            driver.get(f"https://www.google.com/maps/search/{'+'.join(address.split())}")
            wait = WebDriverWait(driver, 10)
            wait.until(EC.url_contains("@"))
            url = driver.current_url
            coords = url.split('@')[1].split(',')[0:2]
            latitude, longitude = float(coords[0]), float(coords[1].split('!')[0])
            cache[address] = {'latitude': latitude, 'longitude': longitude}
            update_cache(cache)  # Atualiza o cache imediatamente após cada geocodificação bem-sucedida
            return latitude, longitude, True
        except Exception as e:
            logging.error(f"Retry {retries + 1} for address '{address}': {e}")
            retries += 1
    return None, None, False

def process_addresses(driver, df_output, cache, address_column_names, output_file):
    for index, row in df_output[df_output['Processed'] == False].iterrows():
        address_parts = [str(row[col]).strip() for col in address_column_names]
        address = ' '.join(address_parts)
        if address and not row['Processed']:
            latitude, longitude, success = get_lat_long(driver, address, cache)
            if success:
                df_output.at[index, 'Latitude'] = latitude
                df_output.at[index, 'Longitude'] = longitude
                df_output.at[index, 'Processed'] = True
                df_output.to_excel(output_file, index=False)  # Salva após cada geocodificação bem-sucedida

    return df_output

def main():
    cache = load_or_create_cache()  # Garante a criação ou carga do arquivo de cache

    input_file = user_input_file()
    df_input = pd.read_excel(input_file)
    address_column_names = user_select_columns(df_input)

    driver = setup_selenium_edge()
    df_output, output_file = prepare_output_file(input_file)

    df_output = process_addresses(driver, df_output, cache, address_column_names, output_file)
    update_cache(cache)
    logging.info("Geocoding process completed.")
    print(f"The spreadsheet has been updated. Output file: {output_file}")

if __name__ == "__main__":
    main()


In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import threading
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import logging
import os

# Setup logging
LOG_FILE = "geocoding.log"
CACHE_FILE = "geocoding_cache.json"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Functions for geocoding
def setup_selenium_edge():
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    logging.info("WebDriver session started.")
    return driver

def load_or_create_cache():
    if not os.path.isfile(CACHE_FILE):
        with open(CACHE_FILE, 'w') as file:
            json.dump({}, file)
        logging.info("New cache file created.")
    try:
        with open(CACHE_FILE, 'r') as file:
            cache = json.load(file)
    except json.JSONDecodeError:
        logging.error("Cache file is corrupted. Creating a new one.")
        cache = {}
    return cache

def update_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4)

def get_lat_long(driver, address, cache, max_retries=3):
    if address in cache:
        return cache[address]['latitude'], cache[address]['longitude'], True
    retries = 0
    while retries < max_retries:
        try:
            driver.get(f"https://www.google.com/maps/search/{'+'.join(address.split())}")
            wait = WebDriverWait(driver, 10)
            wait.until(EC.url_contains("@"))
            url = driver.current_url
            coords = url.split('@')[1].split(',')[0:2]
            latitude, longitude = float(coords[0]), float(coords[1].split('!')[0])
            cache[address] = {'latitude': latitude, 'longitude': longitude}
            update_cache(cache)
            return latitude, longitude, True
        except Exception as e:
            logging.error(f"Retry {retries + 1} for address '{address}': {e}")
            retries += 1
    return None, None, False

def geocode_addresses(input_file, column_indices, progress_callback):
    address_column_names = column_indices.split(',')  # Assuming user enters comma-separated indices
    df_input = pd.read_excel(input_file)
    address_column_names = [df_input.columns[int(index)-1] for index in address_column_names]
    
    driver = setup_selenium_edge()
    cache = load_or_create_cache()
    
    for index, row in df_input.iterrows():
        address_parts = [str(row[col]).strip() for col in address_column_names]
        address = ' '.join(address_parts)
        latitude, longitude, success = get_lat_long(driver, address, cache)
        if success:
            df_input.at[index, 'Latitude'] = latitude
            df_input.at[index, 'Longitude'] = longitude
            progress_callback(index + 1, df_input.shape[0])
    
    update_cache(cache)
    driver.quit()
    
    # Save the modified DataFrame
    output_file = os.path.splitext(input_file)[0] + "_geocoded.xlsx"
    df_input.to_excel(output_file, index=False)
    return output_file

# GUI
class GeocodeApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Geocoding App")
        self.geometry("400x200")
        
        self.create_widgets()
        
    def create_widgets(self):
        self.file_label = tk.Label(self, text="No file selected")
        self.file_label.pack(pady=10)
        
        self.browse_button = tk.Button(self, text="Browse", command=self.browse_file)
        self.browse_button.pack()
        
        self.column_entry_label = tk.Label(self, text="Enter column indices (comma separated):")
        self.column_entry_label.pack(pady=10)
        
        self.column_entry = tk.Entry(self)
        self.column_entry.pack()
        
        self.start_button = tk.Button(self, text="Start Geocoding", command=self.start_geocoding)
        self.start_button.pack(pady=10)
        
        self.progress = ttk.Progressbar(self, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.pack()
        
    def browse_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.file_label.config(text=file_path)
            self.file_path = file_path
    
    def update_progress(self, current, total):
        self.progress['value'] = (current / total) * 100
        self.update_idletasks()
    
    def start_geocoding(self):
        column_indices = self.column_entry.get()
        if not hasattr(self, 'file_path') or not column_indices:
            messagebox.showerror("Error", "Please select a file and enter column indices.")
            return
        threading.Thread(target=geocode_addresses, args=(self.file_path, column_indices, self.update_progress)).start()
        
if __name__ == "__main__":
    app = GeocodeApp()
    app.mainloop()


Exception in thread Thread-5 (geocode_addresses):
Traceback (most recent call last):
  File "C:\Users\jairo\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\jairo\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\jairo\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jairo\AppData\Local\Temp\ipykernel_31784\1516050865.py", line 80, in geocode_addresses
  File "C:\Users\jairo\AppData\Local\Temp\ipykernel_31784\1516050865.py", line 125, in update_progress
  File "C:\Users\jairo\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1713, in __setitem__
    self.configure({key: value})
  File "C:\Users\jairo\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1702, in configure
    return s

In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import threading
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import logging
import os

# Setup logging
LOG_FILE = "geocoding.log"
CACHE_FILE = "geocoding_cache.json"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Global flag to stop geocoding
stop_geocoding = False

def setup_selenium_edge():
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    logging.info("WebDriver session started.")
    return driver

def load_or_create_cache():
    if not os.path.isfile(CACHE_FILE):
        with open(CACHE_FILE, 'w') as file:
            json.dump({}, file)
        logging.info("New cache file created.")
    try:
        with open(CACHE_FILE, 'r') as file:
            cache = json.load(file)
    except json.JSONDecodeError:
        logging.error("Cache file is corrupted. Creating a new one.")
        cache = {}
    return cache

def update_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4)

def get_lat_long(driver, address, cache, max_retries=3):
    if address in cache:
        return cache[address]['latitude'], cache[address]['longitude'], True
    retries = 0
    while retries < max_retries:
        try:
            driver.get(f"https://www.google.com/maps/search/{'+'.join(address.split())}")
            wait = WebDriverWait(driver, 10)
            wait.until(EC.url_contains("@"))
            url = driver.current_url
            coords = url.split('@')[1].split(',')[0:2]
            latitude, longitude = float(coords[0]), float(coords[1].split('!')[0])
            cache[address] = {'latitude': latitude, 'longitude': longitude}
            update_cache(cache)
            return latitude, longitude, True
        except Exception as e:
            logging.error(f"Retry {retries + 1} for address '{address}': {e}")
            retries += 1
    return None, None, False

def geocode_addresses(input_file, column_indices, progress_callback, finished_callback):
    global stop_geocoding
    address_column_names = column_indices.split(',')  # Assuming user enters comma-separated indices
    df_input = pd.read_excel(input_file)
    address_column_names = [df_input.columns[int(index)-1] for index in address_column_names]
    
    driver = setup_selenium_edge()
    cache = load_or_create_cache()
    
    for index, row in df_input.iterrows():
        if stop_geocoding:
            break  # Exit loop if stop signal is received
        address_parts = [str(row[col]).strip() for col in address_column_names]
        address = ' '.join(address_parts)
        latitude, longitude, success = get_lat_long(driver, address, cache)
        if success:
            df_input.at[index, 'Latitude'] = latitude
            df_input.at[index, 'Longitude'] = longitude
            progress_callback(index + 1, df_input.shape[0])
    
    update_cache(cache)
    driver.quit()
    
    output_file = os.path.splitext(input_file)[0] + "_geocoded.xlsx"
    df_input.to_excel(output_file, index=False)
    finished_callback(output_file, not stop_geocoding)

# GUI class with new features...
class GeocodeApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Geocoding App")
        self.geometry("500x300")
        
        self.create_widgets()
        self.file_path = ""
        
    def create_widgets(self):
        self.file_label = tk.Label(self, text="No file selected")
        self.file_label.pack(pady=10)
        
        self.browse_button = tk.Button(self, text="Browse", command=self.browse_file)
        self.browse_button.pack()

        self.columns_label = tk.Label(self, text="")
        self.columns_label.pack(pady=10)
        
        self.column_entry_label = tk.Label(self, text="Enter column indices (comma separated):")
        self.column_entry_label.pack()
        
        self.column_entry = tk.Entry(self)
        self.column_entry.pack()
        
        self.start_button = tk.Button(self, text="Start Geocoding", command=self.start_geocoding)
        self.start_button.pack(pady=5)
        
        self.stop_button = tk.Button(self, text="Stop Geocoding", command=self.stop_geocoding, state=tk.DISABLED)
        self.stop_button.pack(pady=5)

        self.progress_label = tk.Label(self, text="Progress: 0/0")
        self.progress_label.pack(pady=5)
        
        self.progress = ttk.Progressbar(self, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.pack()

    def browse_file(self):
        global stop_geocoding
        stop_geocoding = False  # Reset stop flag on file browse
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if file_path:
            self.file_path = file_path
            self.file_label.config(text=f"File: {os.path.basename(file_path)}")
            df = pd.read_excel(file_path)
            column_text = "Columns: " + ", ".join([f"{i} - {col}" for i, col in enumerate(df.columns, start=1)])
            self.columns_label.config(text=column_text)

    def update_progress(self, current, total):
        self.progress['value'] = (current / total) * 100
        self.progress_label.config(text=f"Progress: {current}/{total}")
        self.update_idletasks()

    def finished_geocoding(self, output_file, success):
        if success:
            messagebox.showinfo("Complete", f"Geocoding complete. Output saved to {output_file}")
        else:
            messagebox.showinfo("Stopped", "Geocoding stopped by user.")
        self.start_button['state'] = tk.NORMAL
        self.stop_button['state'] = tk.DISABLED

    def start_geocoding(self):
        global stop_geocoding
        stop_geocoding = False
        column_indices = self.column_entry.get()
        if not self.file_path or not column_indices:
            messagebox.showerror("Error", "Please select a file and enter column indices.")
            return
        self.start_button['state'] = tk.DISABLED
        self.stop_button['state'] = tk.NORMAL
        threading.Thread(target=geocode_addresses, args=(self.file_path, column_indices, self.update_progress, self.finished_geocoding)).start()

    def stop_geocoding(self):
        global stop_geocoding
        stop_geocoding = True
        self.stop_button['state'] = tk.DISABLED

if __name__ == "__main__":
    app = GeocodeApp()
    app.mainloop()


In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, scrolledtext
import pandas as pd
import threading
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import logging
import os

# Setup logging
LOG_FILE = "geocoding.log"
CACHE_FILE = "geocoding_cache.json"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Global flag to stop geocoding
stop_geocoding = False

def setup_selenium_edge():
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    logging.info("WebDriver session started.")
    return driver

def load_or_create_cache():
    if not os.path.isfile(CACHE_FILE):
        with open(CACHE_FILE, 'w') as file:
            json.dump({}, file)
        logging.info("New cache file created.")
    try:
        with open(CACHE_FILE, 'r') as file:
            cache = json.load(file)
    except json.JSONDecodeError:
        logging.error("Cache file is corrupted. Creating a new one.")
        cache = {}
    return cache

def update_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4)

def get_lat_long(driver, address, cache, max_retries=3):
    if address in cache:
        return cache[address]['latitude'], cache[address]['longitude'], True
    retries = 0
    while retries < max_retries:
        try:
            driver.get(f"https://www.google.com/maps/search/{'+'.join(address.split())}")
            wait = WebDriverWait(driver, 10)
            wait.until(EC.url_contains("@"))
            url = driver.current_url
            coords = url.split('@')[1].split(',')[0:2]
            latitude, longitude = float(coords[0]), float(coords[1].split('!')[0])
            cache[address] = {'latitude': latitude, 'longitude': longitude}
            update_cache(cache)
            return latitude, longitude, True
        except Exception as e:
            logging.error(f"Retry {retries + 1} for address '{address}': {e}")
            retries += 1
    return None, None, False

# Adjusted geocode_addresses to accept a list of column names
def geocode_addresses(input_file, selected_columns, progress_callback, finished_callback):
    global stop_geocoding
    df_input = pd.read_excel(input_file)
    
    driver = setup_selenium_edge()
    cache = load_or_create_cache()
    
    for index, row in df_input.iterrows():
        if stop_geocoding:
            break
        address_parts = [str(row[col]).strip() for col in selected_columns if col in df_input.columns]
        address = ' '.join(address_parts)
        latitude, longitude, success = get_lat_long(driver, address, cache)
        if success:
            df_input.at[index, 'Latitude'] = latitude
            df_input.at[index, 'Longitude'] = longitude
            progress_callback(index + 1, df_input.shape[0])
    
    update_cache(cache)
    driver.quit()
    
    output_file = os.path.splitext(input_file)[0] + "_geocoded.xlsx"
    df_input.to_excel(output_file, index=False)
    finished_callback(output_file, not stop_geocoding)

class GeocodeApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Geocoding App")
        self.geometry("600x400")
        
        self.create_widgets()
        self.file_path = ""
        self.column_checkboxes = []
        self.selected_columns = []
        
    def create_widgets(self):
        self.file_label = tk.Label(self, text="No file selected")
        self.file_label.pack(pady=10)
        
        self.browse_button = tk.Button(self, text="Browse", command=self.browse_file)
        self.browse_button.pack()

        self.columns_frame = tk.LabelFrame(self, text="Columns")
        self.columns_scroll = scrolledtext.ScrolledText(self.columns_frame, width=40, height=10, wrap=tk.WORD)
        self.columns_scroll.pack()
        self.columns_frame.pack(pady=10, fill="both", expand="yes")
        
        self.start_button = tk.Button(self, text="Start Geocoding", command=self.start_geocoding)
        self.start_button.pack(pady=5)
        
        self.stop_button = tk.Button(self, text="Stop Geocoding", command=self.stop_geocoding, state=tk.DISABLED)
        self.stop_button.pack(pady=5)

        self.progress_label = tk.Label(self, text="Progress: 0/0")
        self.progress_label.pack(pady=5)
        
        self.progress = ttk.Progressbar(self, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.pack()

    def browse_file(self):
        global stop_geocoding
        stop_geocoding = False
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if file_path:
            self.file_path = file_path
            self.file_label.config(text=f"File: {os.path.basename(file_path)}")
            self.display_column_checkboxes(file_path)

    def display_column_checkboxes(self, file_path):
        for widget in self.columns_frame.winfo_children():
            widget.destroy()
        
        df = pd.read_excel(file_path)
        self.selected_columns = []
        for col in df.columns:
            var = tk.BooleanVar()
            chk = tk.Checkbutton(self.columns_frame, text=col, variable=var)
            chk.pack(anchor='w')
            self.column_checkboxes.append((var, col))
    
    def update_progress(self, current, total):
        self.progress['value'] = (current / total) * 100
        self.progress_label.config(text=f"Progress: {current}/{total}")
        self.update_idletasks()

    def finished_geocoding(self, output_file, success):
        if success:
            messagebox.showinfo("Complete", f"Geocoding complete. Output saved to {output_file}")
        else:
            messagebox.showinfo("Stopped", "Geocoding stopped by user.")
        self.start_button['state'] = tk.NORMAL
        self.stop_button['state'] = tk.DISABLED

    def start_geocoding(self):
        global stop_geocoding
        stop_geocoding = False
        selected_columns = [col for var, col in self.column_checkboxes if var.get()]
        if not self.file_path or not selected_columns:
            messagebox.showerror("Error", "Please select a file and at least one column.")
            return
        self.start_button['state'] = tk.DISABLED
        self.stop_button['state'] = tk.NORMAL
        threading.Thread(target=geocode_addresses, args=(self.file_path, selected_columns, self.update_progress, self.finished_geocoding)).start()

    def stop_geocoding(self):
        global stop_geocoding
        stop_geocoding = True
        self.stop_button['state'] = tk.DISABLED

if __name__ == "__main__":
    app = GeocodeApp()
    app.mainloop()


In [29]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, scrolledtext
import pandas as pd
import threading
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import logging
import os

# Setup logging
LOG_FILE = "geocoding.log"
CACHE_FILE = "geocoding_cache.json"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Global flag to stop geocoding
stop_geocoding = False

def setup_selenium_edge():
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)
    logging.info("WebDriver session started.")
    return driver

def load_or_create_cache():
    if not os.path.isfile(CACHE_FILE):
        with open(CACHE_FILE, 'w') as file:
            json.dump({}, file)
        logging.info("New cache file created.")
    try:
        with open(CACHE_FILE, 'r') as file:
            cache = json.load(file)
    except json.JSONDecodeError:
        logging.error("Cache file is corrupted. Creating a new one.")
        cache = {}
    return cache

def update_cache(cache):
    with open(CACHE_FILE, 'w') as file:
        json.dump(cache, file, indent=4)

def get_lat_long(driver, address, cache, max_retries=3):
    if address in cache:
        return cache[address]['latitude'], cache[address]['longitude'], True
    retries = 0
    while retries < max_retries:
        try:
            driver.get(f"https://www.google.com/maps/search/{'+'.join(address.split())}")
            wait = WebDriverWait(driver, 10)
            wait.until(EC.url_contains("@"))
            url = driver.current_url
            coords = url.split('@')[1].split(',')[0:2]
            latitude, longitude = float(coords[0]), float(coords[1].split('!')[0])
            cache[address] = {'latitude': latitude, 'longitude': longitude}
            update_cache(cache)
            return latitude, longitude, True
        except Exception as e:
            logging.error(f"Retry {retries + 1} for address '{address}': {e}")
            retries += 1
    return None, None, False

def geocode_addresses(input_file, selected_columns, progress_callback, finished_callback, status_callback):
    global stop_geocoding
    status_callback("Preparing data... This may take some time...")
    df_input = pd.read_excel(input_file)
    
    driver = setup_selenium_edge()
    cache = load_or_create_cache()
    
    status_callback("Geocoding data...")
    for index, row in df_input.iterrows():
        if stop_geocoding:
            break
        address_parts = [str(row[col]).strip() for col in selected_columns if col in df_input.columns]
        address = ' '.join(address_parts)
        latitude, longitude, success = get_lat_long(driver, address, cache)
        if success:
            df_input.at[index, 'Latitude'] = latitude
            df_input.at[index, 'Longitude'] = longitude
            progress_callback(index + 1, df_input.shape[0])
    
    update_cache(cache)
    driver.quit()
    
    output_file = os.path.splitext(input_file)[0] + "_geocoded.xlsx"
    df_input.to_excel(output_file, index=False)
    finished_callback(output_file, not stop_geocoding)

class GeocodeApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Transporte e Meio Ambiente(Trama) - Geocoding with Google - App")
        self.geometry("600x490")  # Increased height for better spacing
        
        self.create_widgets()
        self.file_path = ""
        self.column_checkboxes = []
        self.selected_columns = []
        
    def create_widgets(self):
        self.file_label = tk.Label(self, text="No file selected")
        self.file_label.pack(pady=10)
        
        self.browse_button = tk.Button(self, text="Browse", command=self.browse_file)
        self.browse_button.pack()

        self.columns_frame = tk.LabelFrame(self, text="Columns")
        self.columns_scroll = scrolledtext.ScrolledText(self.columns_frame, width=40, height=10, wrap=tk.WORD)
        self.columns_scroll.pack()
        self.columns_frame.pack(pady=10, fill="both", expand="yes")
        
        self.status_label = tk.Label(self, text="")
        self.status_label.pack(pady=10)

        self.start_button = tk.Button(self, text="Start Geocoding", command=self.start_geocoding)
        self.start_button.pack(pady=5)
        
        self.stop_button = tk.Button(self, text="Stop Geocoding", command=self.stop_geocoding, state=tk.DISABLED)
        self.stop_button.pack(pady=5)

        self.progress_label = tk.Label(self, text="Progress: 0/0")
        self.progress_label.pack(pady=5)
        
        self.progress = ttk.Progressbar(self, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.pack(pady=20)  # Added padding for spacing from the bottom

    def browse_file(self):
        global stop_geocoding
        stop_geocoding = False
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if file_path:
            self.file_path = file_path
            self.file_label.config(text=f"File: {os.path.basename(file_path)}")
            self.display_column_checkboxes(file_path)

    def display_column_checkboxes(self, file_path):
        for widget in self.columns_frame.winfo_children():
            widget.destroy()
        
        df = pd.read_excel(file_path)
        self.selected_columns = []
        for col in df.columns:
            var = tk.BooleanVar()
            chk = tk.Checkbutton(self.columns_frame, text=col, variable=var)
            chk.pack(anchor='w')
            self.column_checkboxes.append((var, col))
    
    def update_progress(self, current, total):
        self.progress['value'] = (current / total) * 100
        self.progress_label.config(text=f"Progress: {current}/{total}")
        self.update_idletasks()

    def finished_geocoding(self, output_file, success):
        if success:
            messagebox.showinfo("Complete", f"Geocoding complete. Output saved to {output_file}")
        else:
            messagebox.showinfo("Stopped", "Geocoding stopped by user.")
        self.start_button['state'] = tk.NORMAL
        self.stop_button['state'] = tk.DISABLED
    
    def update_status(self, message):
        self.status_label.config(text=message)
        self.update_idletasks()

    def start_geocoding(self):
        global stop_geocoding
        stop_geocoding = False
        selected_columns = [col for var, col in self.column_checkboxes if var.get()]
        if not self.file_path or not selected_columns:
            messagebox.showerror("Error", "Please select a file and at least one column.")
            return
        self.start_button['state'] = tk.DISABLED
        self.stop_button['state'] = tk.NORMAL
        threading.Thread(target=geocode_addresses, args=(self.file_path, selected_columns, self.update_progress, self.finished_geocoding, self.update_status)).start()

    def stop_geocoding(self):
        global stop_geocoding
        stop_geocoding = True
        self.stop_button['state'] = tk.DISABLED

if __name__ == "__main__":
    app = GeocodeApp()
    app.mainloop()
